In [1]:
import pyodbc

In [2]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try:
        conn = pyodbc.connect(DRIVER = driver, SERVER= server, DATABASE = database, UID= user, PWD= password)
        print("Successfull")
    except Error as err:
        print(f"Error:'{err}'")
    return conn

In [3]:
pyodbc.drivers()

['SQL Server',
 'PostgreSQL ANSI(x64)',
 'PostgreSQL Unicode(x64)',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

## Windows Authentication

In [4]:
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=master; TRUSTED_CONNECTION=yes;"

In [5]:
conn=pyodbc.connect(conn_string)

## Cursor process

In [6]:
crs = conn.cursor()

In [7]:
conn.autocommit = True

## Create Database

In [8]:
def create_database(conn, create_database_query): #parametreler: connection nesnesi ve query nesnesi
    crs = conn.cursor() #bir cursor nesnesi oluşturuyoruz. bunu query sonucundaki değerleri yakalamak için kullanacağız.
    try:
        crs.execute(create_database_query)#query'yi çalıştır sonucu yakala. queryleri excecute ile çalıştırıyoruz
        print("Database is Created Successfuly")
    except Error as err:
            print(f"Error:'{err}'")

In [9]:
create_database_query = "CREATE DATABASE TestB"

In [10]:
create_database(conn, create_database_query)

Database is Created Successfuly


In [11]:
query = "USE TestB"

In [12]:
crs.execute(query)

In [13]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [14]:
execute_query(conn,query)

Query Succeessful!


In [15]:
query = 'CREATE TABLE TestTable (\
           ID INT IDENTITY (1,1) NOT NULL, \
           FirstName VARCHAR(255) NOT NULL,\
           LastName VARCHAR(255) NOT NULL,\
           PRIMARY KEY(ID))'

In [16]:
execute_query(conn,query)

Query Succeessful!


## Insert

In [17]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES ('Max', 'Maximov')")

In [18]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES ('Maia', 'Mariovna')")

In [19]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?, ?)", 'Jackie', 'Chan')

In [20]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Tom', 'Cat')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Jerry', 'Mouse')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Owen', 'William')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Stefan', 'Müller')

Cursor Function

In [21]:
crs.execute("Delete from TestTable Where FirstName = 'Jerry'")

In [22]:
crs.rowcount

1

## Fetchone

In [23]:
crs.execute("select FirstName, LastName from TestTable")
row = crs.fetchone()
row

('Max', 'Maximov')

## Fetchmany

In [24]:
crs.execute("select FirstName, LastName from TestTable")
row = crs.fetchmany(4)
row

[('Max', 'Maximov'), ('Maia', 'Mariovna'), ('Jackie', 'Chan'), ('Tom', 'Cat')]

## Fetchall

In [25]:
crs.execute("select FirstName, LastName from TestTable")
row = crs.fetchall()
row

[('Max', 'Maximov'),
 ('Maia', 'Mariovna'),
 ('Jackie', 'Chan'),
 ('Tom', 'Cat'),
 ('Owen', 'William'),
 ('Stefan', 'Müller')]

In [26]:
import pandas as pd

In [27]:
crs.execute("select FirstName, LastName from TestTable")
result = crs.fetchall()
result

[('Max', 'Maximov'),
 ('Maia', 'Mariovna'),
 ('Jackie', 'Chan'),
 ('Tom', 'Cat'),
 ('Owen', 'William'),
 ('Stefan', 'Müller')]

In [28]:
df = pd.DataFrame(result)

In [29]:
df

,0
0,"[Max, Maximov]"
1,"[Maia, Mariovna]"
2,"[Jackie, Chan]"
3,"[Tom, Cat]"
4,"[Owen, William]"
5,"[Stefan, Müller]"


In [30]:
a = []
for result in result:
    result=list(result)
    a.append(result)
a

[['Max', 'Maximov'],
 ['Maia', 'Mariovna'],
 ['Jackie', 'Chan'],
 ['Tom', 'Cat'],
 ['Owen', 'William'],
 ['Stefan', 'Müller']]

In [31]:
columns = ['FirstName', 'LastName']
df = pd.DataFrame(a, columns=columns)
df

,FirstName,LastName
0,Max,Maximov
1,Maia,Mariovna
2,Jackie,Chan
3,Tom,Cat
4,Owen,William
5,Stefan,Müller


In [32]:
query = 'USE SampleRetail'
execute_query(conn, query)

Query Succeessful!


In [33]:
df_prod = pd.read_sql("SELECT * from product.product", con=conn)

C:\Users\44778\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [34]:
df_prod.head()

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,DENAQ - AC Adapter for TOSHIBA SATELLITE 1700 ...,23,4,2021,23.99
1,2,NS-SP1800BL 5.1-Channel Home Theater System (B...,5,5,2021,136.99
2,3,Acoustimass 6 Series V Home Theater Speaker Sy...,24,5,2021,599.00
3,4,Corsair Vengeance LPX 16GB (2x8GB) DDR4 DRAM 3...,6,4,2021,151.99
4,5,Details About Samsung Gear Iconx 2018 Edition ...,1,14,2021,199.99


#sql in python